In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
patientFolder = '../input/sample_images'

In [ ]:
%matplotlib inline

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dicom
import os
import scipy.ndimage
from six.moves import cPickle 
import matplotlib.pyplot as plt
import skimage
from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.misc
import itertools

In [ ]:
def getUnlabeledPatients():
    labeledPatients=[]
    unlabeledPatients = []
    for patient in patientList:
        try:
            labels_df.get_value(patient, 'cancer')
            labeledPatients.append(patient)
        except:
            unlabeledPatients.append(patient)
    return labeledPatients,unlabeledPatients

In [ ]:

labels_df = pd.read_csv('../input/stage1_labels.csv', index_col=0)


In [ ]:
patientList = [i for i in os.listdir(patientFolder)]
labeledPatients,unlabeledPatients = getUnlabeledPatients()

In [ ]:
def createDataFrames():
    trainPatients = [labeledPatients[i] for i in Index_Train for j in os.listdir(patientFolder+labeledPatients[i])]
    trainSlices = [j for i in Index_Train for j in os.listdir(patientFolder+labeledPatients[i])]
    trainTarget = [labels_df.get_value(labeledPatients[i], 'cancer') for i in Index_Train for j in os.listdir(patientFolder+labeledPatients[i])]
    testPatients = [labeledPatients[i] for i in Index_Test for j in os.listdir(patientFolder+labeledPatients[i])]
    testSlices = [j for i in Index_Test for j in os.listdir(patientFolder+labeledPatients[i])]
    testTarget = [labels_df.get_value(labeledPatients[i], 'cancer') for i in Index_Test for j in os.listdir(patientFolder+labeledPatients[i])]
    valPatients = [i for i in unlabeledPatients for j in os.listdir(patientFolder+i)]
    valSlices = [j for i in unlabeledPatients for j in os.listdir(patientFolder+i)]
    trainDataFrame = pd.DataFrame({'patientName':trainPatients,'sliceName' : trainSlices,'Target' : trainTarget})
    testDataFrame = pd.DataFrame({'patientName':testPatients,'sliceName' : testSlices,'Target' : testTarget})
    valDataFrame = pd.DataFrame({'patientName':valPatients,'sliceName' : valSlices})
    return trainDataFrame,testDataFrame,valDataFrame

In [ ]:
Index_Train = range(int(0.65*len(labeledPatients)))
Index_Test=[]
for i in range(0,len(labeledPatients)):
    if i in Index_Train:
        continue
    else:
        Index_Test.append(i)
trainDataFrame,testDataFrame,valDataFrame = createDataFrames()

In [ ]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
import numpy as np

from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from six.moves import cPickle 
num_classes = 2
epochs = 50
No_of_slices_in_Train = 5
No_of_slices_in_Val = 7

In [ ]:
input_shape = (224, 224,1)

model = Sequential([
    Conv2D(64, (3, 3), input_shape=input_shape, padding='same',
           activation='relu'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(256, activation='relu'),
    Dense(2, activation='softmax')
])

In [ ]:

def trainBatchGenerator():
    count = 1
    patientNameList = trainDataFrame.patientName.tolist()
    sliceNameList = trainDataFrame.sliceName.tolist()
    while(count<len(sliceNameList)/No_of_slices_in_Val):
        sampleIndex = np.random.choice(len(sliceNameList),No_of_slices_in_Train,replace=False).tolist()
        trainSamplesData = np.stack(cPickle.load(open(patientFolder+patientNameList[i]+"/"+sliceNameList[i], 'rb')) 
                            for i in sampleIndex)
        trainSamplesLabels = [labels_df.get_value(patientNameList[i], 'cancer') for i in sampleIndex]
        trainSamplesLabels = keras.utils.to_categorical(trainSamplesLabels,num_classes)
        trainSamplesData=np.reshape(trainSamplesData, (trainSamplesData.shape[0], 
                                                   trainSamplesData.shape[1], 
                                                   trainSamplesData.shape[2],1))
        count = count+1
        trainSamplesData = trainSamplesData/255 
        yield trainSamplesData,trainSamplesLabels

In [ ]:
if __name__ == "__main__":
    runModel()

In [ ]:
def runModel():
    sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')
    for i in range(1,epochs+1):
        print("*********************** "+str(i)+" epoch ************************")
        for Train_data,Train_labels in trainBatchGenerator():        
                #print("************ " + str(Train_data.shape)+" ********")
                #print("************ " + str(Train_labels.shape)+" ********")
                model.fit(Train_data, Train_labels,nb_epoch=1,
                          batch_size = No_of_slices_in_Train,shuffle=True)
        
        loss=[]
        for valData,valLabels in valBatchGenerator():
            loss.append(model.evaluate(valData, valLabels, batch_size=No_of_slices_in_Val, verbose=1, sample_weight=None))
        print("Validation Loss is ----- "+str(sum(loss)/len(loss)))